In [1]:
import MySQLdb
import sys


In [2]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [79]:
import bz2

def print_logs_configuration(cid):
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT stdlog_file, errlog_file, output_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        stdlog_file, errlog_file, output_file = cursor.fetchone()
            
        if (stdlog_file is None) or (errlog_file is None) or (output_file is None):
            print ("Unable to retrieve cid...")
            return
        
        print("STD LOG FILE")
        print(bz2.decompress(stdlog_file).decode('ascii'))
        print("#########")
        print("ERR LOG FILE")
        print("#########")
        print(bz2.decompress(errlog_file).decode()) # not ascii? 
        print("#########")
        print("OUTPUT LOG FILE")
        print("#########")
        print(bz2.decompress(output_file).decode('ascii'))
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

#print_logs_configuration(116732) #78749: WANXL_BUILD_FIRMWARE # 79052 AIC7XXX_BUILD_FIRMWARE

In [80]:
def get_configuration_file(cid):
    print("Getting configuration file of cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT config_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        config_file = cursor.fetchone()
        
            
        if (config_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            return bz2.decompress(config_file[0]).decode('ascii')
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

cf1 = get_configuration_file(116733) 
#print(cf1)

Getting configuration file of cid= 116733


In [81]:
cf_tiny = get_configuration_file(74458) # same as 126760
ny = 0
for l in cf_tiny.splitlines():
    if l.endswith("=y"):
        ny = ny + 1
        #print(l)
print("number of active options", ny)

Getting configuration file of cid= 74458
number of active options 240


In [5]:
def err_logs_configuration(cid):
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT errlog_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        errlog_file, = cursor.fetchone()
            
        if (errlog_file is None):
            print ("Unable to retrieve cid...")
            return
        
        return bz2.decompress(errlog_file).decode()
    
    except Exception as e:
        print(str(e), "\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

In [14]:
import time
generic_allocator_cid_sample = [65140, 71650, 65543, 73564, 71019, 67934, 75995, 65748, 70818, 64884]
for cid in generic_allocator_cid_sample:
    print("###### CID", cid)
    cf = get_configuration_file(cid)
    # "CONFIG_GENERIC_ALLOCATOR=n" (so not in .config)
    assert("CONFIG_DRM_VBOXVIDEO=y" in cf.splitlines())
    print(err_logs_configuration(cid).splitlines()[:-5])
    print()
    print()
    time.sleep(5)

###### CID 65140
Getting configuration file of cid= 65140
["drivers/gpu/drm/amd/amdgpu/amdgpu_object.c: In function 'amdgpu_bo_create_restricted':", 'drivers/gpu/drm/amd/amdgpu/amdgpu_object.c:378:2: warning: #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance thanks to write-combining [-Wcpp]', ' #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance \\', '  ^~~~~~~', 'drivers/gpu/drm/amd/amdgpu/amdgpu_object.c:378:2: warning: #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance thanks to write-combining [-Wcpp]', ' #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance \\', '  ^~~~~~~', "drivers/gpu/drm/radeon/radeon_object.c: In function 'radeon_bo_create':", 'drivers/gpu/drm/radeon/radeon_object.c:242:2: warning: #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance thanks to write-combining [-Wcpp]', ' #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better 

###### CID 71019
Getting configuration file of cid= 71019
["drivers/gpu/drm/radeon/radeon_object.c: In function 'radeon_bo_create':", 'drivers/gpu/drm/radeon/radeon_object.c:242:2: warning: #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance thanks to write-combining [-Wcpp]', ' #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance \\', '  ^~~~~~~', "drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':", "vbox_hgsmi.c:(.text+0x24): undefined reference to `gen_pool_dma_alloc'", "drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_free':", "vbox_hgsmi.c:(.text+0x105): undefined reference to `gen_pool_free'", "drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_submit':", "vbox_hgsmi.c:(.text+0x11e): undefined reference to `gen_pool_virt_to_phys'", "drivers/staging/vboxvideo/vbox_main.o: In function `vbox_hw_fini':", "vbox_main.c:(.text+0x14a): undefined reference to `gen_pool_destroy'"]


###

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



###### CID 75995
Getting configuration file of cid= 75995


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



###### CID 65748
Getting configuration file of cid= 65748
['In file included from arch/x86/entry/common.c:17:0:', "arch/x86/entry/common.c: In function 'tracehook_report_syscall_exit':", './include/linux/tracehook.h:126:13: note: userspace variable will be forcibly initialized', '   siginfo_t info;', '             ^~~~', "kernel/sysctl_binary.c: In function 'SYSC_sysctl':", 'kernel/sysctl_binary.c:1413:23: note: userspace variable will be forcibly initialized', '  struct __sysctl_args tmp;', '                       ^~~', "kernel/ptrace.c: In function 'ptrace_peek_siginfo':", 'kernel/ptrace.c:712:13: note: userspace variable will be forcibly initialized', '   siginfo_t info;', '             ^~~~', "kernel/ptrace.c: In function 'ptrace_request':", 'kernel/ptrace.c:1076:16: note: userspace variable will be forcibly initialized', '   struct iovec kiov;', '                ^~~~', 'kernel/ptrace.c:890:12: note: userspace variable will be forcibly initialized', '  siginfo_t siginfo, *si;', '  

###### CID 70818
Getting configuration file of cid= 70818
["drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':", "/TuxML/linux-4.13.3/drivers/staging/vboxvideo/vbox_hgsmi.c:76: undefined reference to `gen_pool_dma_alloc'", "drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_free':", "/TuxML/linux-4.13.3/drivers/staging/vboxvideo/vbox_hgsmi.c:101: undefined reference to `gen_pool_free'", "drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_submit':", "/TuxML/linux-4.13.3/drivers/staging/vboxvideo/vbox_hgsmi.c:108: undefined reference to `gen_pool_virt_to_phys'", "drivers/staging/vboxvideo/vbox_main.o: In function `vbox_hw_fini':", "/TuxML/linux-4.13.3/drivers/staging/vboxvideo/vbox_main.c:348: undefined reference to `gen_pool_destroy'"]


###### CID 64884
Getting configuration file of cid= 64884
["drivers/gpu/drm/amd/amdgpu/amdgpu_object.c: In function 'amdgpu_bo_create_restricted':", 'drivers/gpu/drm/amd/amdgpu/amdgpu_object.c:378:2: 

In [15]:
#err_logs_configuration(67934) # very large err log
# err_logs_configuration(75995)

In [16]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC79XX_BUILD_FIRMWARE == 'y'")
# masking effects!
err_logs_configuration(67576)

'aicasm_symbol.c:48:19: fatal error: aicdb.h: No such file or directory\n #include "aicdb.h"\n                   ^\ncompilation terminated.\naicasm_gram.tab.c: In function \'yyparse\':\naicasm_gram.tab.c:1628:16: warning: implicit declaration of function \'yylex\' [-Wimplicit-function-declaration]\naicasm_macro_gram.tab.c: In function \'mmparse\':\n/TuxML/linux-4.13.3/drivers/scsi/aic7xxx/aicasm//aicasm_macro_gram.c:64:25: warning: implicit declaration of function \'mmlex\' [-Wimplicit-function-declaration]\n #define yylex           mmlex\n                         ^\naicasm_macro_gram.tab.c:1199:16: note: in expansion of macro \'yylex\'\naicasm_scan.l: In function \'yylex\':\naicasm_scan.l:420:6: warning: implicit declaration of function \'mm_switch_to_buffer\' [-Wimplicit-function-declaration]\n      yy_switch_to_buffer(temp_state);\n      ^~~~~~~~~~~~~~~~~~~\naicasm_scan.l:421:6: warning: implicit declaration of function \'mmparse\' [-Wimplicit-function-declaration]\n      mm_switch_

In [77]:
#import difflib
#from IPython.core.display import display, HTML

#failure_1 = err_logs_configuration(64884).splitlines()
#failure_2 = err_logs_configuration(65140).splitlines()
#failure_3 = err_logs_configuration(70818).splitlines()
#d = difflib.Differ()
#diff = difflib.ndiff(failure_1, failure_2)
#print('\n'.join(diff))


#display(HTML(difflib.HtmlDiff().make_file(failure_2, failure_3)))



In [78]:
# some cids lead to huuuuge logs 
#generic_allocator_cid_sample2 = [65140, 71650, 65543, 73564, 71019, 65748, 70818, 64884]
#p_failure_log = None
#for cid in generic_allocator_cid_sample2:
#    c_failure_log = err_logs_configuration(cid).splitlines()
#    if (p_failure_log is not None):
#        display(HTML(difflib.HtmlDiff().make_file(p_failure_log, c_failure_log)))
#    p_failure_log = c_failure_log
    

In [14]:
#get_configuration_file(74459)

In [13]:
#get_configuration_file(30698)

In [9]:
#err_logs_configuration(29992)

In [16]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")
# err_logs_configuration(53234) 

In [7]:
err_logs_configuration(59253)

'aicasm_symbol.c:48:19: fatal error: aicdb.h: No such file or directory\n #include "aicdb.h"\n                   ^\ncompilation terminated.\naicasm_gram.tab.c: In function \'yyparse\':\naicasm_gram.tab.c:1628:16: warning: implicit declaration of function \'yylex\' [-Wimplicit-function-declaration]\naicasm_macro_gram.tab.c: In function \'mmparse\':\n/TuxML/linux-4.13.3/drivers/scsi/aic7xxx/aicasm//aicasm_macro_gram.c:64:25: warning: implicit declaration of function \'mmlex\' [-Wimplicit-function-declaration]\n #define yylex           mmlex\n                         ^\naicasm_macro_gram.tab.c:1199:16: note: in expansion of macro \'yylex\'\naicasm_scan.l: In function \'yylex\':\naicasm_scan.l:420:6: warning: implicit declaration of function \'mm_switch_to_buffer\' [-Wimplicit-function-declaration]\n      yy_switch_to_buffer(temp_state);\n      ^~~~~~~~~~~~~~~~~~~\naicasm_scan.l:421:6: warning: implicit declaration of function \'mmparse\' [-Wimplicit-function-declaration]\n      mm_switch_

In [10]:
# err_logs_configuration(61758)

In [12]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")
err_logs_configuration(36364) 

'In file included from ./include/linux/uaccess.h:13:0,\n                 from ./include/linux/highmem.h:8,\n                 from fs/binfmt_elf.c:28:\nfs/binfmt_elf.c: In function \'create_elf_tables\':\n./arch/x86/include/asm/uaccess_64.h:143:20: warning: array subscript is above array bounds [-Warray-bounds]\n    __put_user_asm(4[(u16 *)src], 4 + (u16 __user *)dst,\n                    ^\n./arch/x86/include/asm/uaccess.h:468:16: note: in definition of macro \'__put_user_asm\'\n        : ltype(x), "m" (__m(addr)), "i" (errret), "0" (err))\n                ^\n./arch/x86/include/asm/uaccess_64.h:143:20: warning: array subscript is above array bounds [-Warray-bounds]\n    __put_user_asm(4[(u16 *)src], 4 + (u16 __user *)dst,\n                    ^\n./arch/x86/include/asm/uaccess.h:468:16: note: in definition of macro \'__put_user_asm\'\n        : ltype(x), "m" (__m(addr)), "i" (errret), "0" (err))\n                ^\n./arch/x86/include/asm/uaccess_64.h:154:20: warning: array subscript is 

In [40]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")
for err in err_logs_configuration(50359).splitlines(): #  79099, 59253, 55986, 50359: same
    if "Cyclomatic" not in err:
        print (err)

aicasm_symbol.c:48:19: fatal error: aicdb.h: No such file or directory
 #include "aicdb.h"
                   ^
compilation terminated.
aicasm_gram.tab.c: In function 'yyparse':
aicasm_gram.tab.c:1628:16: warning: implicit declaration of function 'yylex' [-Wimplicit-function-declaration]
aicasm_macro_gram.tab.c: In function 'mmparse':
/TuxML/linux-4.13.3/drivers/scsi/aic7xxx/aicasm//aicasm_macro_gram.c:64:25: warning: implicit declaration of function 'mmlex' [-Wimplicit-function-declaration]
 #define yylex           mmlex
                         ^
aicasm_macro_gram.tab.c:1199:16: note: in expansion of macro 'yylex'
aicasm_scan.l: In function 'yylex':
aicasm_scan.l:420:6: warning: implicit declaration of function 'mm_switch_to_buffer' [-Wimplicit-function-declaration]
      yy_switch_to_buffer(temp_state);
      ^~~~~~~~~~~~~~~~~~~
aicasm_scan.l:421:6: warning: implicit declaration of function 'mmparse' [-Wimplicit-function-declaration]
      mm_switch_to_buffer(old_state);
      ^~~~~

In [41]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")
err_logs_configuration(99909)

"drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':\nvbox_hgsmi.c:(.text+0x24): undefined reference to `gen_pool_dma_alloc'\ndrivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_free':\nvbox_hgsmi.c:(.text+0x105): undefined reference to `gen_pool_free'\ndrivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_submit':\nvbox_hgsmi.c:(.text+0x11e): undefined reference to `gen_pool_virt_to_phys'\ndrivers/staging/vboxvideo/vbox_main.o: In function `vbox_hw_fini':\nvbox_main.c:(.text+0x14a): undefined reference to `gen_pool_destroy'\ndrivers/staging/vboxvideo/vbox_main.o: In function `vbox_driver_load':\nvbox_main.c:(.text+0x561): undefined reference to `gen_pool_create'\nvbox_main.c:(.text+0x58e): undefined reference to `gen_pool_add_virt'\nvbox_main.c:(.text+0x59e): undefined reference to `gen_pool_destroy'\nmake: *** [vmlinux] Error 1\n"

In [68]:
# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y')")
#err_logs_configuration(86309) # strange since it seems we have two errors basically (the compilation process do not stop or restart based on apt-file?)
# err_logs_configuration(72724) # same here
# err_logs_configuration(62069) # same
# err_logs_configuration(42176)
# err_logs_configuration(35796)
# err_logs_configuration(70886)
# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values
#for cid in [32606, 33318, 34546, 35049, 35796, 36425, 37814, 39355, 40657,
#       41365, 41938, 42176, 42589, 44158, 45350, 45829, 46217, 46707,
#       48523, 49378, 52069, 52558, 53111, 56475, 56600, 57646, 57813,
#       58899, 58963, 59414, 59653, 59970, 60569, 62533, 63122, 63173,
#       63470, 63834, 67713, 67945, 68049, 68300, 69288, 69532, 72676,
#       72724, 74223, 74238, 76652, 77528, 77620, 77885, 77988, 78244,
#       78544, 79010, 80924, 80941, 81315, 82382, 83619, 83702, 84325]:
#    print("configuration cid", cid)
#    for err in err_logs_configuration(cid).splitlines(): #  79099, 59253, 55986, 50359: same    
#        if "as68k" in err:
#            print ("as68k", err)
# err_logs_configuration(80924) 

# cids_as68k = [] # rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")['cid'].values
#    as68k_found = False
#    for err in err_logs_configuration(cid).splitlines(): #  79099, 59253, 55986, 50359: same    
#        if "as68k" in err:
#            as68k_found = True
#    if not as68k_found:
#        print("configuration has no trace of as68k", cid)
        

In [45]:
# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").sample(n=10)['cid'].values
# err_logs_configuration(80206) # strange one since the error message suggests the root cause is elsewhere
err_logs_configuration(31560)

"drivers/gpu/drm/radeon/radeon_object.c: In function 'radeon_bo_create':\ndrivers/gpu/drm/radeon/radeon_object.c:242:2: warning: #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance thanks to write-combining [-Wcpp]\n #warning Please enable CONFIG_MTRR and CONFIG_X86_PAT for better performance \\\n  ^~~~~~~\n/bin/sh: 1: as68k: not found\nmake[3]: *** [drivers/net/wan/wanxlfw.inc] Error 127\nmake[2]: *** [drivers/net/wan] Error 2\nmake[1]: *** [drivers/net] Error 2\nmake[1]: *** Waiting for unfinished jobs....\nmake: *** [drivers] Error 2\n"

In [76]:
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values
#cids_generic_allocator = [30901, 37341, 40757, 48193, 50359, 55986, 59253, 79099]
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")['cid'].values
#cids_generic_allocator = [36364, 53234] 
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")['cid'].values
cids_generic_allocator = [98510, 98511, 98902, 98920, 99203, 99287, 99909] 
for cid in cids_generic_allocator:
    print("configuration cid", cid)
    for err in err_logs_configuration(cid).splitlines(): #  79099, 59253, 55986, 50359: same    
        if "Cyclomatic" not in err:
            print (err)
    print()
    print()
    

configuration cid 98510
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':
vbox_hgsmi.c:(.text+0x24): undefined reference to `gen_pool_dma_alloc'
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_free':
vbox_hgsmi.c:(.text+0x105): undefined reference to `gen_pool_free'
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_submit':
vbox_hgsmi.c:(.text+0x11e): undefined reference to `gen_pool_virt_to_phys'
drivers/staging/vboxvideo/vbox_main.o: In function `vbox_hw_fini':
vbox_main.c:(.text+0x14a): undefined reference to `gen_pool_destroy'
drivers/staging/vboxvideo/vbox_main.o: In function `vbox_driver_load':
vbox_main.c:(.text+0x565): undefined reference to `gen_pool_create'
vbox_main.c:(.text+0x592): undefined reference to `gen_pool_add_virt'
vbox_main.c:(.text+0x6da): undefined reference to `gen_pool_destroy'
make: *** [vmlinux] Error 1


configuration cid 98511
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':

drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_alloc':
vbox_hgsmi.c:(.text+0x24): undefined reference to `gen_pool_dma_alloc'
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_free':
vbox_hgsmi.c:(.text+0x105): undefined reference to `gen_pool_free'
drivers/staging/vboxvideo/vbox_hgsmi.o: In function `hgsmi_buffer_submit':
vbox_hgsmi.c:(.text+0x11e): undefined reference to `gen_pool_virt_to_phys'
drivers/staging/vboxvideo/vbox_main.o: In function `vbox_hw_fini':
vbox_main.c:(.text+0x14a): undefined reference to `gen_pool_destroy'
drivers/staging/vboxvideo/vbox_main.o: In function `vbox_driver_load':
vbox_main.c:(.text+0x552): undefined reference to `gen_pool_create'
vbox_main.c:(.text+0x57f): undefined reference to `gen_pool_add_virt'
vbox_main.c:(.text+0x58f): undefined reference to `gen_pool_destroy'
make: *** [vmlinux] Error 1


configuration cid 99287
drivers/gpu/drm/amd/amdgpu/amdgpu_object.c: In function ‘amdgpu_bo_create_restricted’:
drivers/gp